In [26]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import json
import re
from bs4 import BeautifulSoup as bs

In [34]:
options = webdriver.FirefoxOptions()
driver = webdriver.Firefox(options=options)
driver.get("https://www.indeed.com/jobs?q=data+science&l=remote")
time.sleep(1)

soup = bs(driver.page_source, features='html.parser')

In [35]:
results = soup.find(id="jobsearch-Main")

In [36]:
job_elements = soup.find_all("div", class_="slider_item")

In [38]:
basePath = "https://www.indeed.com"
d = {}
# Currently using link_element_id as unique ID.  I am fairly sure it's unique - at least it has been from what I've seen so far.
for job_element in job_elements[:1]:
    link_element_id = job_element.find_all("a")[0]["id"]

    d[link_element_id] = {}
    d[link_element_id]["title"] = job_element.find("h2", class_="jobTitle").text.strip()
    d[link_element_id]['company'] = job_element.find("span", class_="companyName").text.strip()
    d[link_element_id]['posted'] = job_element.find("span", class_="date").next_element.next_element.next_element
    try:
        d[link_element_id]['location'] = job_element.find("div", class_="companyLocation").text.strip()
    except AttributeError: 
        d[link_element_id]['location'] = "Not Listed"
    try:
        d[link_element_id]['salary'] = job_element.find("div", class_="metadata").text.strip()
    except AttributeError:
        d[link_element_id]['salary'] = "Not Listed"
    # date_element = job_element.find("div", class_="visually-hidden")  # not useful as-is.  figure out if this is possible to find at all

    WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="{link_element_id}"]'))).click()
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="jobDescriptionText"]')))
    temp_results = results.find("div", id="jobDescriptionText")
    # print(d[link_element_id]['title'], d[link_element_id]['company'])
    print(d)
    print(temp_results.text.strip())
    d[link_element_id]['desc'] = temp_results.text.strip()

{'sj_885612f252daab9d': {'title': 'Security Research Manager (Data Science)', 'company': 'Engtal', 'posted': 'Active 2 days ago', 'location': 'Remote', 'salary': '$200,000 - $225,000 a year'}}
Security Research Manager (Data)Remote, US $225k Salary + Excellent Package
I am partnered exclusively with an exciting technology leader in the Cyber Security space, searching for a Security Research Manager as part of a brand new position within the Product team.
This industry leader is redefining how organizations handle exposure, performance, and risk, both for themselves and their partners. Businesses place their trust in this innovative company to guide their cybersecurity investments, foster trust within their network, and mitigate financial vulnerabilities.
The Product Research team works directly with the groups varied customer base to define user driven technology and ensure the organizations methodologies focus on solving key challenges within the industry. The Security Research Manage

In [39]:
d

{'sj_885612f252daab9d': {'title': 'Security Research Manager (Data Science)',
  'company': 'Engtal',
  'posted': 'Active 2 days ago',
  'location': 'Remote',
  'salary': '$200,000 - $225,000 a year',
  'desc': 'Security Research Manager (Data)Remote, US $225k Salary + Excellent Package\nI am partnered exclusively with an exciting technology leader in the Cyber Security space, searching for a Security Research Manager as part of a brand new position within the Product team.\nThis industry leader is redefining how organizations handle exposure, performance, and risk, both for themselves and their partners. Businesses place their trust in this innovative company to guide their cybersecurity investments, foster trust within their network, and mitigate financial vulnerabilities.\nThe Product Research team works directly with the groups varied customer base to define user driven technology and ensure the organizations methodologies focus on solving key challenges within the industry. The Sec

Once I can get the full job desc, I will want to do the following:
For each job, create a JSON object for it.  Each JSON object will have a structure like

{ "title":job_title, "company":company_name, "location":job_location, "salary":job_salary, "snippet":job_desc_snippet, "desc":full_job_desc, "tags":search_terms_used_to_find_job}

I want a way to go through and if the same job is repeated, add on to the tags in the proper json.  Can do this by instead building it as a dict, appending to the "tags" key so then search_terms_used_to_find_job is instead a list thereof, and the list is appended with the new search terms before being json-ified.

Need to figure out if I can / how to get the date posted.

Need to figure out if the job ID on indeed is unique.  If I can't figure it out, may need my own unique ID for each

Probably remove the job desc snippet since I have full descs now

Need to figure out how to go through all pages

Need to add tags for search params

Can use: https://scrapfly.io/blog/how-to-scrape-indeedcom/
To see how to better optimize this search.  Both for making searching more dynamic as well as using async IO to speed it up